## 쿨루프 시공 대상 여부 분류
- Tasks
    1. Data Preprocessing
        - 모델 사용을 위해 데이터를 일관성 있게 정리해야 합니다.
    2. Object Detection
        - 전처리 된 데이터를 이용하여 학습을 진행하세요.
        - 인공위성 지도를 이용하여 이미지를 **추가적으로** 10장 수집하고 추론 과정에서 사용하세요.

In [ ]:
!pip install keras --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.2 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1


In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [ ]:
import gdown, zipfile
import os, glob, shutil
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt

### 데이터셋 다운로드 및 압축 해제
- cool_roof_image.zip : 이미지 데이터 압축 파일
- cool_roof_yolo_labels.zip : YOLO 모델 형식에 맞게 가공된 레이블 파일

In [ ]:
def dataset_extract(file_name):
    with zipfile.ZipFile(file_name, 'r') as zip_ref :
        file_list = zip_ref.namelist()

        if os.path.exists(f'/content/{file_name[-14:-4]}'):
            print(f'데이터셋 폴더가 이미 존재합니다.')
            return

        else :
            for f in tqdm(file_list, desc='Extracting', unit='files') :
                zip_ref.extract(member=f, path=f'/content/{file_name[-14:-4]}/')


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_extract('/content/drive/MyDrive/미니프로젝트4차/cool_roof.v1i.yolov8-obb.zip')

Extracting: 100%|██████████| 708/708 [00:00<00:00, 775.06files/s] 


In [ ]:
# dataset_extract('/content/drive/MyDrive/미니프로젝트4차/cool_roof_yolo_labels.zip')

Extracting: 100%|██████████| 204/204 [00:00<00:00, 971.64files/s]


### 폴더 생성
- YOLO 모델에서 요구하는 폴더의 형식이 있습니다.
- 해당 형식에 맞춰 폴더를 만드세요.

### 데이터 스플릿 & 파일 이동
1. 재현을 위한 난수 고정 : 2024
2. 테스트셋 데이터 20%
- 위 설정에 맞게 데이터를 나누고, 위 과정에서 생성한 폴더에 이동시키세요.

In [ ]:
import os
import random
from sklearn.model_selection import train_test_split
import shutil

# 이미지 파일이 들어있는 폴더 경로
folder_path = '/content/roof/oof_images'
train_path = '/content/roof/train/images'
valid_path = '/content/roof/valid/images'

# 이미지 파일 목록 가져오기
file_list = os.listdir(folder_path)

# train 데이터셋과 test 데이터셋으로 나누기
train_files, valid_files = train_test_split(file_list, test_size=0.2, random_state=42)

# train 데이터셋 이동
for file in train_files:
    shutil.copy(os.path.join(folder_path, file), os.path.join(train_path, file))

# test 데이터셋 이동
for file in valid_files:
    shutil.copy(os.path.join(folder_path, file), os.path.join(valid_path, file))


FileNotFoundError: [Errno 2] No such file or directory: '/content/roof/oof_images'

In [ ]:
len(train_files), len(valid_files)

(160, 40)

In [ ]:
import os
import shutil

image_folder = '/content/roof/train/images'
label_folder = '/content/roof/olo_labels/obj_train_data'
label_path = '/content/roof/train/labels'

image_files = [file for file in os.listdir(image_folder) if file.endswith('.jpg')]
label_files = [file for file in os.listdir(label_folder) if file.endswith('.txt')]

for image_file in image_files:
    label_file = image_file.replace('.jpg', '.txt')
    if label_file in label_files:
        src = os.path.join(label_folder, label_file)
        dst = os.path.join(label_path, label_file)
        shutil.copy(src, dst)


In [ ]:
import os
import shutil

image_folder = '/content/roof/valid/images'
label_folder = '/content/roof/olo_labels/obj_train_data'
label_path = '/content/roof/valid/labels'

image_files = [file for file in os.listdir(image_folder) if file.endswith('.jpg')]
label_files = [file for file in os.listdir(label_folder) if file.endswith('.txt')]

for image_file in image_files:
    label_file = image_file.replace('.jpg', '.txt')
    if label_file in label_files:
        src = os.path.join(label_folder, label_file)
        dst = os.path.join(label_path, label_file)
        shutil.copy(src, dst)


### YOLO 모델에 적용할 YAML 생성하기
- 지붕에는 두 가지 형태가 있습니다. 클래스 구분에 주의하세요.
- cool roof
- generic roof

In [ ]:
import yaml

data = {
    "train" : '/content/yolov8-obb/cool_roof.v1i.yolov8-obb/train/images',
        "val" : '/content/yolov8-obb/cool_roof.v1i.yolov8-obb/valid/images',
        # "test" : '/tld_sample/test/', optional
        "names" : {1 : 'generic_roof', 0 : 'cool_roof'}}

with open('./roof.yaml', 'w') as f :
    yaml.dump(data, f)

# check written file
with open('./roof.yaml', 'r') as f :
    lines = yaml.safe_load(f)
    print(lines)

{'names': {0: 'cool_roof', 1: 'generic_roof'}, 'train': '/content/yolov8-obb/cool_roof.v1i.yolov8-obb/train/images', 'val': '/content/yolov8-obb/cool_roof.v1i.yolov8-obb/valid/images'}


In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 401.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.8/871.8 kB 18.6 MB/s eta 0:00:00


### YOLO v8 모델
- yaml 파일의 경로 설정에 주의하세요.

In [ ]:
from ultralytics import YOLO, settings

In [ ]:
model = YOLO(model='yolov8n-obb.pt')

100%|██████████| 6.26M/6.26M [00:00<00:00, 87.0MB/s]


In [ ]:
model.train(data='/content/roof.yaml',
            epochs=250,
            patience=80,
            pretrained=True,
            verbose=True,
            seed=2023,
            )

Ultralytics YOLOv8.2.91 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=obb, mode=train, model=yolov8n-obb.pt, data=/content/roof.yaml, epochs=250, time=None, patience=80, batch=16, imgsz=1024, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=2023, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_box

100%|██████████| 755k/755k [00:00<00:00, 21.3MB/s]


Overriding model.yaml nc=15 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 6.25M/6.25M [00:00<00:00, 93.7MB/s]


AMP: checks passed ✅


...

                   all         28        324      0.659      0.834      0.786      0.564



250 epochs completed in 1.174 hours.
Optimizer stripped from runs/obb/train/weights/last.pt, 7.1MB
Optimizer stripped from runs/obb/train/weights/best.pt, 7.1MB

Validating runs/obb/train/weights/best.pt...
Ultralytics YOLOv8.2.91 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-obb summary (fused): 187 layers, 3,077,609 parameters, 0 gradients, 8.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.27it/s]


                   all         28        324       0.71      0.821      0.817      0.606
             cool_roof         23         48      0.727      0.875      0.868      0.668
          generic_roof         28        276      0.693      0.768      0.767      0.544
Speed: 0.5ms preprocess, 6.1ms inference, 0.0ms loss, 8.7ms postprocess per image
Results saved to runs/obb/train


ultralytics.utils.metrics.OBBMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f6625723160>
curves: []
curves_results: []
fitness: 0.6268259887321336
keys: ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)']
maps: array([     0.6677,     0.54359])
names: {0: 'cool_roof', 1: 'generic_roof'}
plot: True
results_dict: {'metrics/precision(B)': 0.7096380976297347, 'metrics/recall(B)': 0.8213968524958863, 'metrics/mAP50(B)': 0.8174659469374737, 'metrics/mAP50-95(B)': 0.6056437711537624, 'fitness': 0.6268259887321336}
save_dir: PosixPath('runs/obb/train')
speed: {'preprocess': 0.47194106238228933, 'inference': 6.128507001059396, 'loss': 0.0017455645969935826, 'postprocess': 8.652261325291224}

In [ ]:
results = model.predict(source='/content/test_img/18_png.rf.4234236a492ccd50badc11fda3668b7f.jpg', conf=0.7, iou=0.3,
                        save=True,
                        line_width=2)


image 1/1 /content/test_img/18_png.rf.4234236a492ccd50badc11fda3668b7f.jpg: 1024x1024 14.1ms
Speed: 5.3ms preprocess, 14.1ms inference, 3.0ms postprocess per image at shape (1, 3, 1024, 1024)
Results saved to runs/obb/train2


In [ ]:
results = model.predict(source='/content/yolov8-obb/cool_roof.v1i.yolov8-obb/test/images/18_png.rf.4234236a492ccd50badc11fda3668b7f.jpg', conf=0.7, iou=0.3,
                        save=True,
                        line_width=2)


image 1/1 /content/yolov8-obb/cool_roof.v1i.yolov8-obb/test/images/18_png.rf.4234236a492ccd50badc11fda3668b7f.jpg: 1024x1024 14.1ms
Speed: 9.1ms preprocess, 14.1ms inference, 3.6ms postprocess per image at shape (1, 3, 1024, 1024)
Results saved to runs/obb/train2


In [ ]:
results = model.predict(source='/content/yolov8-obb/cool_roof.v1i.yolov8-obb/test/images/5-1-_png.rf.7920d69eb88c65daacb95038858fffc7.jpg', conf=0.7, iou=0.3,
                        save=True,
                        line_width=2)


image 1/1 /content/yolov8-obb/cool_roof.v1i.yolov8-obb/test/images/5-1-_png.rf.7920d69eb88c65daacb95038858fffc7.jpg: 1024x1024 14.2ms
Speed: 8.8ms preprocess, 14.2ms inference, 3.6ms postprocess per image at shape (1, 3, 1024, 1024)
Results saved to runs/obb/train3


In [ ]:
/content/yolov8-obb/cool_roof.v1i.yolov8-obb/test/images/5-1-_png.rf.7920d69eb88c65daacb95038858fffc7.jpg